In [ ]:
!pip install -q transformers datasets scikit-learn accelerate


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    set_seed
)
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_GT/tweets-train.csv")
val_df = pd.read_csv("/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_GT/tweets-valid.csv")
test_df = pd.read_csv('/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_GT/tweets-test.csv')
train_df.head()


,tweet,label,political
0,होता होता राहीलेला निवडणूक मारो मर्ज़ीभई,-1,0
1,खरा लखोबा तर हा बोबडाच आहे,-1,-1
2,मुंबईतील घाटकोपरमध्ये धुळवड खेळून घरी परतलेलं ...,-1,0
3,वेडाबाई भूतकाळ बघ लोक शेन घालतात आणी दांडा ही,-1,-1
4,कुर्ला वाहतुक विभागाला फक्त हे पाठवले जाते पण ...,-1,-1


In [ ]:
# Drop rows with missing tweet or label
train_df = train_df.dropna(subset=["tweet", "label"])
val_df   = val_df.dropna(subset=["tweet", "label"])
test_df  = test_df.dropna(subset=["tweet", "label"])

# Convert labels from (-1, 0, 1) to (0, 1, 2)
for df in [train_df, val_df, test_df]:
    df["label"] = df["label"].astype(int)
    df["label"] = df["label"] + 1  # -1->0, 0->1, 1->2

train_df["label"].value_counts(), val_df["label"].value_counts(), test_df["label"].value_counts()


(label
 0    4000
 2    4000
 1    3998
 Name: count, dtype: int64,
 label
 0    500
 1    500
 2    500
 Name: count, dtype: int64,
 label
 0    500
 1    500
 2    500
 Name: count, dtype: int64)

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset   = Dataset.from_pandas(val_df)
test_dataset  = Dataset.from_pandas(test_df)


In [ ]:
model_name = "l3cube-pune/marathi-bert-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/455 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
def tokenize_fn(batch):
    return tokenizer(
        batch["tweet"],
        truncation=True,
        padding="max_length",
        max_length=128
    )


In [ ]:
train_encoded = train_dataset.map(tokenize_fn, batched=True)
val_encoded   = val_dataset.map(tokenize_fn, batched=True)
test_encoded  = test_dataset.map(tokenize_fn, batched=True)


Map:   0%|          | 0/11998 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
def cleanup_columns(ds):
    cols_to_keep = ["input_ids", "attention_mask", "label"]
    cols_to_remove = [c for c in ds.column_names if c not in cols_to_keep]
    return ds.remove_columns(cols_to_remove)

train_encoded = cleanup_columns(train_encoded)
val_encoded   = cleanup_columns(val_encoded)
test_encoded  = cleanup_columns(test_encoded)

print(train_encoded)


Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 11998
})


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3
)


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/951M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/marathi-bert-v2 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "precision_macro": precision_score(labels, preds, average="macro", zero_division=0),
        "recall_macro": recall_score(labels, preds, average="macro", zero_division=0),
        "f1_macro": f1_score(labels, preds, average="macro", zero_division=0),
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="./gt_marathi_bert_v2",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_steps=50,
    fp16=True,             # if your GPU supports it
    report_to="none"
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encoded,
    eval_dataset=val_encoded,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


/tmp/ipython-input-1429388747.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,Precision Macro,Recall Macro,F1 Macro
1,0.438400,0.552766,0.793333,0.795708,0.793333,0.794057
2,0.404800,0.545758,0.798000,0.796227,0.798000,0.796708
3,0.324400,0.628605,0.790667,0.792465,0.790667,0.791205


TrainOutput(global_step=2250, training_loss=0.38397616238064236, metrics={'train_runtime': 562.0375, 'train_samples_per_second': 64.042, 'train_steps_per_second': 4.003, 'total_flos': 2367626089425408.0, 'train_loss': 0.38397616238064236, 'epoch': 3.0})

In [ ]:
test_metrics = trainer.evaluate(test_encoded)
print("Test metrics:", test_metrics)


Test metrics: {'eval_loss': 0.5473583936691284, 'eval_accuracy': 0.788, 'eval_precision_macro': 0.7863243003973945, 'eval_recall_macro': 0.7879999999999999, 'eval_f1_macro': 0.7868957944039545, 'eval_runtime': 3.0679, 'eval_samples_per_second': 488.937, 'eval_steps_per_second': 30.64, 'epoch': 3.0}
